<h1>12  Introduction to Modeling Libraries in Python</h1>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision=4, suppress=True)
PREVIOUS_MAX_ROWS = pd.options.display.max_rows

<p>In this book, I have focused on providing a programming foundation for doing data analysis in Python. Since data analysts and scientists often report spending a disproportionate amount of time with data wrangling and preparation, the book's structure reflects the importance of mastering these techniques.</p>

<p>Which library you use for developing models will depend on the application. Many statistical problems can be solved by simpler techniques like ordinary least squares regression, while other problems may call for more advanced machine learning methods. Fortunately, Python has become one of the languages of choice for implementing analytical methods, so there are many tools you can explore after completing this book.</p>

<p>In this chapter, I will review some features of pandas that may be helpful when you're crossing back and forth between data wrangling with pandas and model fitting and scoring. I will then give short introductions to two popular modeling toolkits, <a href="http://statsmodels.org/">statsmodels</a> and <a href="http://scikit-learn.org/">scikit-learn</a>. Since each of these projects is large enough to warrant its own dedicated book, I make no effort to be comprehensive and instead direct you to both projects' online documentation along with some other Python-based books on data science, statistics, and machine learning.</p>

<h2>12.1 Interfacing Between pandas and Model Code</h2>

<p>A common workflow for model development is to use pandas for data loading and cleaning before switching over to a modeling library to build the model itself. An important part of the model development process is called <em>feature engineering</em> in machine learning. This can describe any data transformation or analytics that extract information from a raw dataset that may be useful in a modeling context. The data aggregation and GroupBy tools we have explored in this book are used often in a feature engineering context.</p>

<p>While details of "good" feature engineering are out of scope for this book, I will show some methods to make switching between data manipulation with pandas and modeling as painless as possible.</p>

<p>The point of contact between pandas and other analysis libraries is usually NumPy arrays. To turn a DataFrame into a NumPy array, use the <code>to_numpy</code> method:</p>

In [2]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [3]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [4]:
data.to_numpy()

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

<p>To convert back to a DataFrame, as you may recall from earlier chapters, you can pass a two-dimensional ndarray with optional column names:</p>

In [5]:
df2 = pd.DataFrame(data.to_numpy(), columns=['one', 'two', 'three'])
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


<p>The <code>to_numpy</code> method is intended to be used when your data is homogeneous—for example, all numeric types. If you have heterogeneous data, the result will be an ndarray of Python objects:</p>

In [6]:
df3 = data.copy()
df3['strings'] = ['a', 'b', 'c', 'd', 'e']
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [7]:
df3.to_numpy()

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

<p>For some models, you may wish to use only a subset of the columns. I recommend using <code>loc</code> indexing with <code>to_numpy</code>:</p>

In [8]:
model_cols = ['x0', 'x1']
data.loc[:, model_cols].to_numpy()

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

<p>Some libraries have native support for pandas and do some of this work for you automatically: converting to NumPy from DataFrame and attaching model parameter names to the columns of output tables or Series. In other cases, you will have to perform this "metadata management" manually.</p>

<p>In Ch 7.5: Categorical Data, we looked at pandas's <code>Categorical</code> type and the <code>pandas.get_dummies</code> function. Suppose we had a nonnumeric column in our example dataset:</p>

In [9]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                  categories=['a', 'b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


<p>If we wanted to replace the <code>'category'</code> column with dummy variables, we create dummy variables, drop the <code>'category'</code> column, and then join the result:</p>

In [10]:
dummies = pd.get_dummies(data.category, prefix='category')
data_with_dummies = data.drop('category', axis=1).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


<p>There are some nuances to fitting certain statistical models with dummy variables. It may be simpler and less error-prone to use Patsy (the subject of the next section) when you have more than simple numeric columns.</p>

<h2>12.2 Creating Model Descriptions with Patsy</h2>

<p><a href="https://patsy.readthedocs.io/">Patsy</a> is a Python library for describing statistical models (especially linear models) with a string-based "formula syntax," which is inspired by (but not exactly the same as) the formula syntax used by the R and S statistical programming languages. It is installed automatically when you install statsmodels:</p>

<pre>conda install statsmodels</pre>

<p>Patsy is well supported for specifying linear models in statsmodels, so I will focus on some of the main features to help you get up and running. Patsy's formulas are a special string syntax that looks like:</p>

<pre>y ~ x0 + x1</pre>

<p>The syntax <code>a + b</code> does not mean to add <code>a</code> to <code>b</code>, but rather that these are terms in the design matrix created for the model. The <code>patsy.dmatrices</code> function takes a formula string along with a dataset (which can be a DataFrame or a dictionary of arrays) and produces design matrices for a linear model:</p>

In [11]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [12]:
import patsy
y, X = patsy.dmatrices('y ~ x0 + x1', data)

<p>Now we have:</p>

In [13]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [14]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

<p>These Patsy <code>DesignMatrix</code> instances ar NumPy ndarrays with additional metadata:</p>

In [16]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [15]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

<p>You might wonder where the <code>Intercept</code> term came from. This is a convention for linear models like ordinary least squares (OLS) regression. You can suppress the intercept by adding the term <code>+ 0</code> to the model:</p>

In [17]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

<p>The Patsy objects can be passed directly into algorithms like <code>numpy.linalg.lstsq</code>, which performs an ordinary least squares regression:</p>

In [21]:
coef, resid, _, _ = np.linalg.lstsq(X, y, rcond=None)

<p>The model metadata is retained in the <code>design_info</code> attribute, so you can reattach the model column names to the fitted coefficients to obtain a Series, for example:</p>

In [22]:
coef

array([[ 0.3129],
       [-0.0791],
       [-0.2655]])

In [23]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

<h3>Data Transformations in Patsy Formulas</h3>

<p>You can mix Python code into your Patsy formulas; when evaluating the formula, the library will try to find the functions you use in the enclosing scope:</p>

In [24]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

<p>Some commonly used variable transformations include <em>standardizing</em> (to mean 0 and variance 1) and <em>centering</em> (subtracting the mean). Patsy has built-in functions for this purpose:</p>

In [25]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

<p>As part of a modeling process, you may fit a model on one dataset, then evaluate the model based on another. This might be a hold-out portion or new data that is observed later. When applying transformations like center and standardize, you should be careful when using the model to form predications based on new data. These are called <em>stateful</em> transformations, because you must use statistics like the mean or standard deviation of the original dataset when transforming a new dataset.</p>

<p>The <code>patsy.build_design_matrices</code> function can apply transformations to new <code>out-of-sample</code> data using the saved information from the original <code>in-sample</code> dataset:</p>

In [26]:
new_data = pd.DataFrame({
    'x0': [6, 7, 8, 9],
    'x1': [3.1, -0.5, 0, 2.3],
    'y': [1, 2, 3, 4]})
new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

<p>Because the plus symbol (<code>+</code>) in the context of Patsy formulas does not mean addition, when you want to add columns from a dataset by name, you must wrap them in the special <code>I</code> function:</p>

In [27]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

<p>Patsy has several other built-in transforms in the <code>patsy.builtins</code> module. See the online documentation for more.</p>

<p>Categorical data has a special class of transformations, which I explain next.</p>

<h3>Categorical Data and Patsy</h3>

<p>Nonnumeric data can be transformed for a model design matrix in many different ways. A complete treatment of this topic is outside the scope of this book and would be studied best along with a course in statistics.</p>

<p>When you use nonnumeric terms in a Patsy formula, they are converted to dummy variables by default. If there is an intercept, one of the levels will be left out to avoid collinearity:</p>

In [28]:
data = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})
y, X = patsy.dmatrices('v2 ~ key1', data)
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

<p>If you omit the intercept from the model, then columns for each category value will be included in the model design matrix:</p>

In [29]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

<p>Numeric columns can be interpreted as categorical with the C function:</p>

In [30]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

<p>When you're using multiple categorical terms in a model, things can be more complicated, as you can include interaction terms of the form <code>key1:key2</code>, which can be used, for example, in analysis of variance (ANOVA) models:</p>

In [31]:
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [32]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [33]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

<p>Patsy provides for other ways to transform categorical data, including transformations for terms with a particular ordering. See the online documentation for more.</p>

<h2>12.3 Introduction to statsmodels</h2>

<p><a href="http://www.statsmodels.org/">statsmodels</a> is a Python library for fitting many kinds of statistical models, performing statistical tests, and data exploration and visualization. statsmodels contains more "classical" frequentist statistical methods, while Bayesian methods and machine learning models are found in other libraries.</p>

<p>Some kinds of models found in statsmodels include:</p>
<ul>
    <li>Linear models, generalized linear models, and robust linear models</li>
    <li>Linear mixed effects models</li>
    <li>Analysis of variance (ANOVA) methods</li>
    <li>Time series processes and state space models</li>
    <li>Generalized method of moments</li>
</ul>
<p>In the next few pages, we will use a few basic tools in statsmodels and explore how to use the modeling interfaces with Patsy formulas and pandas DataFrame objects. If you didn't install statsmodels in the Patsy discussion earlier, you can install it now with:</p>

<pre>conda install statsmodels</pre>

<h3>Estimating Linear Models</h3>

<p>There are several kinds of linear regression models in statsmodels, from the more basic (e.g., ordinary least squares) to more complex (e.g., iteratively reweighted least squares).</p>

<p>Linear models in statsmodels have two different main interfaces: array based and formula based. These are accessed through these API module imports:</p>

In [34]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

<p>To show how to use these, we generate a linear model from some random data. Run the following code in a Jupyter cell:</p>

In [35]:
# To make the example reproducible
rng = np.random.default_rng(seed=12345)

def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * rng.standard_normal(*size)

N = 100
X = np.c_[dnorm(0, 0.4, size=N), #concatenate along 2nd axis
          dnorm(0, 0.6, size=N),
          dnorm(0, 0.2, size=N)]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

<p>Here, I wrote down the "true" model with known parameters <code>beta</code>. In this case, <code>dnorm</code> is a helper function for generating normally distributed data with a particular mean and variance. So now we have:</p>

In [36]:
X[:5]

array([[-0.9005, -0.1894, -1.0279],
       [ 0.7993, -1.546 , -0.3274],
       [-0.5507, -0.1203,  0.3294],
       [-0.1639,  0.824 ,  0.2083],
       [-0.0477, -0.2131, -0.0482]])

In [37]:
y[:5]

array([-0.5995, -0.5885,  0.1856, -0.0075, -0.0154])

<p>A linear model is generally fitted with an intercept term, as we saw before with Patsy. The <code>sm.add_constant</code> function can add an intercept column to an existing matrix:</p>

In [38]:
X_model = sm.add_constant(X)
X_model[:5]

array([[ 1.    , -0.9005, -0.1894, -1.0279],
       [ 1.    ,  0.7993, -1.546 , -0.3274],
       [ 1.    , -0.5507, -0.1203,  0.3294],
       [ 1.    , -0.1639,  0.824 ,  0.2083],
       [ 1.    , -0.0477, -0.2131, -0.0482]])

<p>The <code>sm.OLS</code> class can fit an ordinary least squares linear regression:</p>

In [39]:
model = sm.OLS(y, X)

<p>The model's <code>fit</code> method returns a regression results object containing estimated model parameters and other diagnostics:</p>

In [40]:
results = model.fit()
results.params

array([0.0668, 0.268 , 0.4505])

<p>The <code>summary</code> method on <code>results</code> can print a model detailing diagnostic output of the model:</p>

In [41]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.469
Model:                            OLS   Adj. R-squared (uncentered):              0.452
Method:                 Least Squares   F-statistic:                              28.51
Date:                Wed, 12 Apr 2023   Prob (F-statistic):                    2.66e-13
Time:                        13:48:34   Log-Likelihood:                         -25.611
No. Observations:                 100   AIC:                                      57.22
Df Residuals:                      97   BIC:                                      65.04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0668      0.054      1.243      0.217      -0.040       0.174
x2             0.2680      0.042      6.313      0.000       0.184       0.352
x3             0.4505      0.068      6.605      0.000       0.315       0.586
==============================================================================
Omnibus:                        0.435   Durbin-Watson:                   1.869
Prob(Omnibus):                  0.805   Jarque-Bera (JB):                0.301
Skew:                           0.134   Prob(JB):                        0.860
Kurtosis:                       2.995   Cond. No.                         1.64
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<p>The parameter names here have been given the generic names <code>x1</code>, <code>x2</code>, and so on. Suppose instead that all of the model parameters are in a DataFrame:</p>

In [42]:
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])
data['y'] = y
data[:5]

,col0,col1,col2,y
0,-0.900506,-0.189430,-1.027870,-0.599527
1,0.799252,-1.545984,-0.327397,-0.588454
2,-0.550655,-0.120254,0.329359,0.185634
3,-0.163916,0.824040,0.208275,-0.007477
4,-0.047651,-0.213147,-0.048244,-0.015374


<p>Now we can use the statsmodel formula API and Patsy formula strings:</p>

In [43]:
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()
results.params

Intercept   -0.020799
col0         0.065813
col1         0.268970
col2         0.449419
dtype: float64

In [44]:
results.tvalues

Intercept   -0.652501
col0         1.219768
col1         6.312369
col2         6.567428
dtype: float64

<p>Observe how statsmodels has returned results as Series with the DataFrame column names attached. We also do not need to use <code>add_constant</code> when using formulas and pandas objects.</p>

<p>Given new out-of-sample data, you can compute predicted values given the estimated model parameters:</p>

In [45]:
results.predict(data[:5])

0   -0.592959
1   -0.531160
2    0.058636
3    0.283658
4   -0.102947
dtype: float64

<p>There are many additional tools for analysis, diagnostics, and visualization of linear model results in statsmodels that you can explore. There are also other kinds of linear models beyond ordinary least squares.</p>

<h3>Estimating Time Series Processes</h3>

<p>Another class of models in statsmodels is for time series analysis. Among these are autoregressive processes, Kalman filtering and other state space models, and multivariate autoregressive models.</p>

<p>Let's simulate some time series data with an autoregressive structure and noise. Run the following in Jupyter:</p>

In [46]:
init_x = 4

values = [init_x, init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

<p>This data has an AR(2) structure (two <code>lags</code>) with parameters <code>0.8</code> and <code>–0.4</code>. When you fit an AR model, you may not know the number of lagged terms to include, so you can fit the model with some larger number of lags:</p>

In [47]:
from statsmodels.tsa.ar_model import AutoReg
MAXLAGS = 5
model = AutoReg(values, MAXLAGS)
results = model.fit()

<p>The estimated parameters in the results have the intercept first, and the estimates for the first two lags next:</p>

In [48]:
results.params

array([ 0.0235,  0.8097, -0.4287, -0.0334,  0.0427, -0.0567])

<p>Deeper details of these models and how to interpret their results are beyond what I can cover in this book, but there's plenty more to discover in the statsmodels documentation.</p>

<h3>12.4 Introduction to scikit-learn</h3>

<p><a href="http://scikit-learn.org/">scikit-learn</a> is one of the most widely used and trusted general-purpose Python machine learning toolkits. It contains a broad selection of standard supervised and unsupervised machine learning methods, with tools for model selection and evaluation, data transformation, data loading, and model persistence. These models can be used for classification, clustering, prediction, and other common tasks. You can install scikit-learn from conda like so:</p>

<pre>conda install scikit-learn</pre>

<p>There are excellent online and print resources for learning about machine learning and how to apply libraries like scikit-learn to solve real-world problems. In this section, I will give a brief flavor of the scikit-learn API style.</p>

<p>pandas integration in scikit-learn has improved significantly in recent years, and by the time you are reading this it may have improved even more. I encourage you to check out the latest project documentation.</p>

<p>As an example for this chapter, I use a <a href="https://www.kaggle.com/c/titanic">now-classic dataset from a Kaggle competition</a> about passenger survival rates on the Titanic in 1912. We load the training and test datasets using pandas:</p>

In [49]:
train = pd.read_csv('datasets/titanic/train.csv')
test = pd.read_csv('datasets/titanic/test.csv')
train.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


<p>Libraries like statsmodels and scikit-learn generally cannot be fed missing data, so we look at the columns to see if there are any that contain missing data:</p>

In [50]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [51]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

<p>In statistics and machine learning examples like this one, a typical task is to predict whether a passenger would survive based on features in the data. A model is fitted on a <em>training</em> dataset and then evaluated on an out-of-sample <em>testing</em> dataset.</p>

<p>I would like to use <code>Age</code> as a predictor, but it has missing data. There are a number of ways to do missing data imputation, but I will do a simple one and use the median of the training dataset to fill the nulls in both tables:</p>

In [52]:
impute_value = train['Age'].median()
train['Age'] = train['Age'].fillna(impute_value)
test['Age'] = test['Age'].fillna(impute_value)

<p>Now we need to specify our models. I add a column <code>IsFemale</code> as an encoded version of the <code>'Sex'</code> column:</p>

In [53]:
train['IsFemale'] = (train['Sex'] == 'female').astype(int) #cast
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

<p>Then we decide on some model variables and create NumPy arrays:</p>

In [54]:
predictors = ['Pclass', 'IsFemale', 'Age']

X_train = train[predictors].to_numpy()
X_test = test[predictors].to_numpy()
y_train = train['Survived'].to_numpy()
X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [55]:
y_train[:5]

array([0, 1, 1, 1, 0])

<p>I make no claims that this is a good model or that these features are engineered properly. We use the <code>LogisticRegression</code> model from scikit-learn and create a model instance:</p>

In [57]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

<p>We can fit this model to the training data using the model's <code>fit</code> method:</p>

In [60]:
model.fit(X_train, y_train)

LogisticRegression()

<p>Now, we can form predictions for tthe test dataset using <code>model.predict</code>:</p>

In [61]:
y_predict = model.predict(X_test)
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0])

<p>If you had the true values for the test dataset, you could compute an accuracy percentage or some other error metric:</p>

<pre>(y_true == y_predict).mean()</pre>

<p>In practice, there are often many additional layers of complexity in model training. Many models have parameters that can be tuned, and there are techniques such as cross-validation that can be used for parameter tuning to avoid overfitting to the training data. This can often yield better predictive performance or robustness on new data.</p>

<p>Cross-validation works by splitting the training data to simulate out-of-sample prediction. Based on a model accuracy score like mean squared error, you can perform a grid search on model parameters. Some models, like logistic regression, have estimator classes with built-in cross-validation. For example, the <code>LogisticRegressionCV</code> class can be used with a parameter indicating how fine-grained of a grid search to do on the model regularization parameter <code>C</code>:</p>


In [62]:
from sklearn.linear_model import LogisticRegressionCV
model_cv = LogisticRegressionCV(Cs=10)
model_cv.fit(X_train, y_train)

LogisticRegressionCV()

<p>To do cross-validation by hand, you can use the <code>cross_val_score</code> helper function, which handles the data splitting process. For example, to cross-validate our model with four nonoverlapping splits of the training data, we can do:</p>

In [63]:
from sklearn.model_selection import cross_val_score
model = LogisticRegression(C=10)
scores = cross_val_score(model, X_train, y_train, cv=4)
scores

array([0.7758, 0.7982, 0.7758, 0.7883])

<p>The default scoring metric is model dependent, but it is possible to choose an explicit scoring function. Cross-validated models take longer to train but can often yield better model performance.</p>

<h2>12.5 Conclusion</h2>

<p>While I have only skimmed the surface of some Python modeling libraries, there are more and more frameworks for various kinds of statistics and machine learning either implemented in Python or with a Python user interface.</p>

<p>This book is focused especially on data wrangling, but there are many others dedicated to modeling and data science tools. Some excellent ones are:</p>
<ul>
    <li>Introduction to Machine Learning with Python by Andreas Müller and Sarah Guido (O'Reilly)</li>
    <li>Python Data Science Handbook by Jake VanderPlas (O'Reilly)</li>
    <li>Data Science from Scratch: First Principles with Python by Joel Grus (O'Reilly)</li>
    <li>Python Machine Learning by Sebastian Raschka and Vahid Mirjalili (Packt Publishing)</li>
    <li>Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow by Aurélien Géron (O'Reilly)</li>
</ul>
<p>While books can be valuable resources for learning, they can sometimes grow out of date when the underlying open source software changes. It's a good idea to be familiar with the documentation for the various statistics or machine learning frameworks to stay up to date on the latest features and API.</p>

In [46]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS